In [ ]:
!pip install pyportfolioopt --quiet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%matplotlib inline

from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import plotting


In [ ]:
# Read monthly data 
dfAll = pd.read_csv('AllMonthlyData.csv', parse_dates = ["Date"], index_col = "Date", dtype=np.float64)
dfAll.fillna(method='ffill', inplace=True)
dfAll.head()

In [ ]:
# Calculate exponential moving average mean historical return 
mu = expected_returns.ema_historical_return(dfAll, frequency=250, span=180, log_returns=True)

# Covariance matrix 
S = risk_models.exp_cov(dfAll, returns_data=False, span=180, frequency=250, log_returns=True)

# Correlation matrix
R = risk_models.cov_to_corr(S)
print('Correlation Matrix:','\r\n',R)

fig, ax = plt.subplots(figsize=(10,15))

ef = EfficientFrontier(mu, S, weight_bounds=(0,1))
plotting.plot_efficient_frontier(ef, ef_param='return', show_assets=False)

ef_minvol=EfficientFrontier(mu, S, weight_bounds=(0,1))
ef_maxS =EfficientFrontier(mu,S,weight_bounds=(0,1))

minret = ef_minvol.min_volatility()
maxS = ef_maxS.max_sharpe()

ret_tangent, std_tangent, slp = ef.portfolio_performance(verbose=False, risk_free_rate=0.0153)
y_vol, x_vol,slope = ef_minvol.portfolio_performance(verbose=False,risk_free_rate=0.0153)

# Generate random portfolios
n_samples = 10000
w = np.random.dirichlet(np.ones(len(mu)), n_samples)
rets = w.dot(mu)
stds = np.sqrt(np.diag(w @ S @ w.T))
sharpes = rets / stds

# Output
a = ax.scatter(stds, rets, marker=".", s=1, c=sharpes, cmap='Wistia')
b = ax.scatter(std_tangent, ret_tangent, marker=".", s=200, c="red", label="Max Sharpe")
c = ax.scatter(x_vol, y_vol, marker="^",s=150, c='blue', label='Min Vol')
ax.set_title("Efficient Frontier with random portfolios")
ax.legend()
plt.tight_layout()
plt.colorbar(a)

print('')
print('Min Volatility Weights:',ef_minvol.clean_weights())
print('')
print('Max Sharpe Weights:', ef.clean_weights())
print('')